<a href="https://colab.research.google.com/github/basselkassem/nlp-toolkit/blob/master/4_language_models_perplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk.corpus import reuters
from nltk import trigrams
from nltk.util import ngrams
from collections import Counter, defaultdict
from itertools import chain
import numpy as np

In [2]:
import nltk
nltk.download('reuters')
nltk.download('punkt')

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# ngram Language models

Language models assign a probabilites to the sequences: $word_1word_2...word_k$. Those models can be used to predict the next word given the previous context.

Predicting the upcomming word, or giving a probability to a sentence is important in the following task:

*   text completing / suggestions in messaging apps
*   spelling correction
*   machine translation(what is the most probable translation among all potential translations)

**Build the model**

$p(w_1, w_2, ..., w_k) = p(w_1)p(w_2|w_1)p(w_3|w_2,w_1)...p(w_k|w_{k-1},...w_1)$
* add $<start>$ token and $<end>$ token to have a probability distribution
* Markov assumption: $p(w_n|w_{k-1},...w_1) =p(w_n|w_{k-1})$
* use ngram to serve as a context for the predicted word

$p(w_1, w_2, ..., w_k) = \prod_{i=1}^{k+1}p(w_i|w_{i-n + 1}^{i-1})$ where $w_{i-n + 1}^{i-1} = (w_{i-n+1}...w_{i-1})$ i.e the previous ngram 

**Training:**

Training can be done using loglikehood estimation:

$p(w_i|w_{i-n + 1}^{i-1}) = \frac{\#(w_i,w_{i-n + 1}^{i-1})}{\#(w_{i-n + 1}^{i-1})}= \frac{\#(w_{i-n + 1}^{i})}{\#(w_{i-n + 1}^{i-1})}$



In [65]:
corpus =list(reuters.sents())
np.random.shuffle(corpus)
print(len(corpus))

54716


In [0]:
model = defaultdict(lambda: defaultdict(lambda: 0))

#count occurences of w3 given w2, w1
for sentence in corpus:
  for w1, w2, w3 in trigrams(sentence, pad_right = True, pad_left = True):
    model[(w1, w2)][w3] += 1

#normalize count of w3 given w2, w1 to probabilities
for w1_w2 in model:
  norm = float(sum(model[w1_w2].values()))
  for w in model[w1_w2]:
    model[w1_w2][w] /= norm

In [69]:
def generate_text(beginnig, model):
  is_finished = False
  text = beginnig.split()

  while not is_finished:
    w1_w2 = tuple(text[-2:])
    thershold = np.random.random()
    score = 0.0
    for w3 in model[w1_w2].keys():
      score += model[w1_w2][w3]
      if score >= thershold:
        text.append(w3)
        break
    if text[-2:] == [None, None]:
      is_finished = True
  return ' '.join(list(filter(None, text)))

for i in range(10):
  print(generate_text('money is', model))

money is a leader in the quarter and 2 . 3 mln
money is holding discussions with other debt .
money is being carried at " international markets , officials said .
money is a major source of employment contracts with pay television networks said on his interest in a submission to the bankruptcy court ruling against it and Millipore intend to bring U . K ., West Germany .
money is wasted .
money is a political commitment to 275 dlrs per share .
money is holding up well from now ?"
money is a rising trend in the index for input prices showed a surplus of 1 . 71 mln .
money is a dramatic reversal in the U . S . Norstar operates in Canada ' s industry aid package that goes beyond the reach of our fiscal 1987 ' s posted price for the new division ' s central bank intervention to the International Cocoa Organization , ICO , passed over us ," he said .
money is a private placement , which has ruled out offering to sell interests in some parts and recombining parts with other railroads , adds t

# Perplexity

It is clear that bigger ngrams gives more accurate probabilites on the train set. But using bigger ngrams makes it harder to predict on the test set. In other words, it is more likely that the model will find a matching between 2-grams on the test set and the training set than finding a matching when n-gram = 7.

Perplexity compute the quality of our language model by computing:

$likelihood = p(w_{test}) = \prod_{i=1}^{N +1}p(w_i|w_{i-n+1}^{i-1})$

$perplexity = \sqrt[N]\frac{1}{p(w_{test})}$

The smaller the perplexity, the better the model is

In [70]:
train_txt = 'This is the rat that ate the malt that lay in the house that Jack built'
test_txt = 'This is the house that Jack built'
train_txt = train_txt.lower()
test_txt = test_txt.lower()
print(train_txt)
print(test_txt)

this is the rat that ate the malt that lay in the house that jack built
this is the house that jack built


In [0]:
def create_start_tokens(n_of_ngram, start_token = 'start'):
  start_tokens = []
  for i in range(n_of_ngram - 1):
    start_tokens.append('<' + start_token + str(i + 1)  + '>')
  return start_tokens

def create_end_tokens(end_token = 'end'):
  end_token = '<' + end_token + '>'
  return end_token

def pad_text(text, n_of_ngram, start_token = 'start', end_token = 'end'):
  start_tokens = create_start_tokens(n_of_ngram, start_token)
  end_token = create_end_tokens(end_token)
  return ' '.join(start_tokens) + ' ' + text + ' ' + end_token

In [72]:
n_of_ngram = 3
train = pad_text(train_txt, n_of_ngram)
test = pad_text(test_txt, n_of_ngram)
print(train)
print(test)

<start1> <start2> this is the rat that ate the malt that lay in the house that jack built <end>
<start1> <start2> this is the house that jack built <end>


In [0]:
def create_ngrams(text, n_of_ngram):
  n_grams = ngrams(text.split(), n_of_ngram)
  return [' '.join(grams) for grams in  n_grams]

In [74]:
train_unigrams = create_ngrams(train, n_of_ngram = 1)
test_unigrams = create_ngrams(test, n_of_ngram = 1)
print(len(train_unigrams), len(test_unigrams))

train_bigrams = create_ngrams(train, n_of_ngram = 2)
test_bigrams = create_ngrams(test, n_of_ngram = 2)
print(len(train_bigrams), len(test_bigrams))

train_trigrams = create_ngrams(train, n_of_ngram = 3)
test_trigrams = create_ngrams(test, n_of_ngram = 3)
print(len(train_trigrams), len(test_trigrams))

19 10
18 9
17 8


In [75]:
unique_train_unigrams = set(train_unigrams)
start_tokens = create_start_tokens(n_of_ngram)
for token in start_tokens:
  unique_train_unigrams -= {token}
vocab_size = len(unique_train_unigrams)
N_test = len(test_txt.split())
print(vocab_size, N_test)

13 7


# Build language model

In [0]:
unigrams_count = Counter(train_unigrams)
bigrams_count = Counter(train_bigrams)
trigrams_count = Counter(train_trigrams)

In [0]:
def compute_liklehood(ngrams, n_of_ngram):
  liklehood = 1
  for wi_wi_1_wi_2 in ngrams:
    tokens = wi_wi_1_wi_2.split()
    wi_1_wi_2 = ' '.join(tokens[: -(n_of_ngram - 1) + 1])
    nominator = trigrams_count[wi_wi_1_wi_2]
    dominator = bigrams_count[wi_1_wi_2]
    liklehood *=  nominator / dominator
  return liklehood

In [86]:
liklehood = compute_liklehood(test_trigrams, n_of_ngram)
print(liklehood)
#print(compute_perplexity(liklehood))

0.0


# add-one-smoothing

In [0]:
def compute_liklehood_add_one(ngrams, n_of_ngram):
  liklehood = 1
  for wi_wi_1_wi_2 in ngrams:
    tokens = wi_wi_1_wi_2.split()
    wi_1_i_2 = ' '.join(tokens[: -(n_of_ngram - 1) + 1])

    nominator = trigrams_count[wi_wi_1_wi_2] + 1
    dominator = bigrams_count[wi_1_i_2] + vocab_size
    liklehood *=  nominator / dominator
  return liklehood

def compute_perplexity(liklehood):
  return np.power(liklehood, -1 / N_test)

In [88]:
liklehood = compute_liklehood_add_one(test_trigrams, n_of_ngram)
perplexity = compute_perplexity(liklehood)
print(perplexity)

10.205413747033983


# interpolate-smoothing

In [0]:
def compute_liklehood_interpolate(ngrams, n_of_ngram):
  lamb1 = lamb2 = lamb3 = 1.0 / 3
  assert lamb1 + lamb2 + lamb3 == 1
  liklehood = 1
  for wi_2_wi_1_wi in ngrams:
    tokens = wi_2_wi_1_wi.split()
    # p(wi given wi-2, wi-1)
    wi_2_wi_1 = ' '.join(tokens[: -(n_of_ngram - 1) + 1])
    p_wi_given_wi_2_wi_1 = lamb1 * trigrams_count[wi_2_wi_1_wi] / bigrams_count[wi_2_wi_1]

    # p(wi given wi-1)
    wi_1 = ' '.join(tokens[-(n_of_ngram - 1): -(n_of_ngram - 1) + 1])
    wi_wi_1 = ' '.join(tokens[-(n_of_ngram - 1): ])
    p_wi_given_wi_1 = lamb2 * bigrams_count[wi_wi_1] / unigrams_count[wi_1]

    # p(wi)
    wi = tokens[-1]
    p_wi = lamb3 * 1 / unigrams_count[wi]

    p_hat = p_wi_given_wi_2_wi_1 + p_wi_given_wi_1 + p_wi
    liklehood *= p_hat
  return liklehood
liklehood = compute_liklehood_interpolate(test_trigrams, n_of_ngram)

In [90]:
perplexity = compute_perplexity(liklehood)
print(perplexity)

1.2505123624523526
